# Import modules

In [62]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

    100% |████████████████████████████████| 81kB 1.6MB/s 
  Running setup.py bdist_wheel for simplejson ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Successfully built simplejson


In [60]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_data = """query data ($nationality: String!, $job: String!)
            {movieList(nationality: $nationality, job: $job)
                {
                title 
                revenue
                }}"""
query_introspect = """{
  __schema {
    types {
      name
    }
  }
}"""
query_playground = """{"operationName":"IntrospectionQuery","variables":{},"query":"query IntrospectionQuery {\n  __schema {\n    queryType {\n      name\n    }\n    mutationType {\n      name\n    }\n    subscriptionType {\n      name\n    }\n    types {\n      ...FullType\n    }\n    directives {\n      name\n      description\n      locations\n      args {\n        ...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n  kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n    description\n    args {\n      ...InputValue\n    }\n    type {\n      ...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\n  kind\n  name\n  ofType {\n    kind\n    name\n    ofType {\n      kind\n      name\n      ofType {\n        kind\n        name\n        ofType {\n          kind\n          name\n          ofType {\n            kind\n            name\n            ofType {\n              kind\n              name\n              ofType {\n                kind\n                name\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n}\n"}"""

In [61]:
input_query_data = json.dumps({'variables': {'nationality': 'English', 'job': 'Director'}, 'query':query_data})
input_query_metadata = json.dumps({'query':query_metadata})

In [64]:
simplejson.loads(query_playground)

JSONDecodeError: Invalid control character '\n' at: line 1 column 89 (char 88)

# Test handler functions

In [35]:
import handler 
importlib.reload(handler);

In [45]:
%%time
handler.handle(input_query_data)

CPU times: user 26.9 ms, sys: 0 ns, total: 26.9 ms
Wall time: 26 ms


'{"movieList": [{"title": "Madagascar 3: Europes Most Wanted", "revenue": 746921274}, {"title": "Mamma Mia!", "revenue": 609841637}, {"title": "Madagascar: Escape 2 Africa", "revenue": 603900354}]}'

In [46]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.json'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [47]:
function_url = 'https://labs.cloud.contiamo.com/b0a002bf-d166-5ccc-a552-ebe89882cb90/v1/function/be852eaa-c701-4f73-8ed7-aceb6dd4aeb8/index_by_nationality'

In [55]:
%%time
deployed_output = requests.post(function_url,
                                data=query_playground).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [53]:
deployed_output

{'__schema': {'queryType': {'name': 'Query'},
  'mutationType': None,
  'subscriptionType': None,
  'types': [{'kind': 'OBJECT',
    'name': 'Query',
    'description': None,
    'fields': [{'name': 'nationalityList',
      'description': None,
      'args': [],
      'type': {'kind': 'NON_NULL',
       'name': None,
       'ofType': {'kind': 'LIST',
        'name': None,
        'ofType': {'kind': 'SCALAR', 'name': 'String', 'ofType': None}}},
      'isDeprecated': False,
      'deprecationReason': None},
     {'name': 'jobList',
      'description': None,
      'args': [],
      'type': {'kind': 'NON_NULL',
       'name': None,
       'ofType': {'kind': 'LIST',
        'name': None,
        'ofType': {'kind': 'SCALAR', 'name': 'String', 'ofType': None}}},
      'isDeprecated': False,
      'deprecationReason': None},
     {'name': 'movieList',
      'description': None,
      'args': [{'name': 'nationality',
        'description': None,
        'type': {'kind': 'SCALAR', 'name': 'Str

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True